In [7]:
// https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps

#r "nuget: FSharp.Data,6.0.1-beta001"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.ImageExport"

open FSharp.Data
open System
open Plotly.NET
open Plotly.NET.ImageExport

[<Literal>]
let ResolutionFolder = __SOURCE_DIRECTORY__

type Despesas = CsvProvider<"./despesa_ceaps_2022.csv", ";", ResolutionFolder = ResolutionFolder>

let despesas = Despesas.Load(ResolutionFolder + "/despesa_ceaps_2022.csv")

let Senador = "MARCOS DO VAL"

let parseDecimal (value:String) = Decimal.Parse(value.Replace(',','.'))

let parseDate (value:String) = DateTime.ParseExact(value, "dd/MM/yyyy", null)

let monthly (value:String) =
    let date = parseDate value
    date.ToString("MM/yyyy")

let data = 
    despesas.Rows
    |> Seq.filter(fun d -> d.SENADOR = Senador && d.ANO = 2022)

Installed Packages FSharp.Data, 6.0.1-beta001 Plotly.NET.ImageExport, 3.0.1 Plotly.NET.Interactive, 3.0.2

In [8]:
let dates, sums =
    data
    |> Seq.groupBy(fun it -> monthly it.DATA)
    |> Seq.map(fun (date, values) -> date, (values |> Seq.sumBy(fun it -> parseDecimal it.VALOR_REEMBOLSADO)))
    |> Seq.sortBy fst 
    |> Seq.toList
    |> List.unzip
Chart.Line(dates, sums)

Error: System.ArgumentException: No formatter is registered for MIME type MM/yyyy.
   at Microsoft.DotNet.Interactive.Formatting.Formatter.InferPreferredFormatter(Type actualType, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 559
   at Microsoft.DotNet.Interactive.Formatting.Formatter.<>c__DisplayClass52_0.<GetPreferredFormatterFor>b__0(ValueTuple`2 tuple) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 520
   at System.Collections.Concurrent.ConcurrentDictionary`2.GetOrAdd(TKey key, Func`2 valueFactory)
   at Microsoft.DotNet.Interactive.Formatting.Formatter.GetPreferredFormatterFor(Type actualType, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter.cs:line 514
   at Microsoft.DotNet.Interactive.Formatting.Formatter`1.FormatTo(T obj, FormatContext context, String mimeType) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\Formatter{T}.cs:line 58
  

In [2]:
data
|> Seq.groupBy(fun it -> it.FORNECEDOR)
|> Seq.map( fun (supplier, expenses) ->
    let dates, values =
        expenses
        |> Seq.map(fun it -> ( parseDate it.DATA), (it.VALOR_REEMBOLSADO |> parseDecimal))
        |> Seq.sortBy fst
        |> Seq.toArray
        |> Array.unzip
    Chart.Point(dates, values, Name = supplier)
)
|> Chart.combine
|> Chart.withSize(1400, 1000)
|> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
|> Chart.withTitle("Despesas com cota parlamentar " + Senador)


<!-- Plotly chart will be drawn inside this DIV -->